# Qualche esempio di uso dei transformer 

Luca Mari, settembre 2024  

Quest'opera è distribuita con <a href="http://creativecommons.org/licenses/by-nc-sa/4.0" target="_blank">Licenza Creative Commons Attribuzione - Non commerciale - Condividi allo stesso modo 4.0 Internazionale</a>.  
<img src="https://creativecommons.it/chapterIT/wp-content/uploads/2021/01/by-nc-sa.eu_.png" width="100">

**Obiettivo**: essere informati di alcune possibili modalità di uso "di alto livello" dei trasformer per applicazioni con testi in lingua italiana e della libreria Python `transformers` di Hugging Face.  
**Precompetenze**: basi di Python.

> Per eseguire questo notebook con VSCode sul proprio calcolatore, occorre:
> * installare un interprete Python
> * scaricare da https://code.visualstudio.com/download e installare VSCode
> * eseguire VSCode e attivare le estensioni per Python e Jupyter
> * ancora in VSCode:
>     * creare una cartella di lavoro e renderla la cartella corrente
>     * copiare nella cartella il file di questa attività: [transformers.ipynb](transformers.ipynb)
>     * aprire il notebook `transformers.ipynb`
>     * creare un ambiente virtuale locale Python (Select Kernel | Python Environments | Create Python Environment | Venv, e scegliere un interprete Python):
>     * installare i moduli Python richiesti, eseguendo dal terminale:  
>         `pip install transformer torch`

Sappiamo che con sistemi come `ChatGPT` possiamo dialogare liberamente e in molte lingue. Grazie al modulo Python `transformers` di Hugging Face (https://huggingface.co/docs/transformers) e ai tanti modelli accessibili e scaricabili liberamente da Hugging Face stesso, con poche linee di codice Python è possibile costruire applicazioni con funzionalità specifiche, anche in grado di trattare testi in lingua italiana con una qualità spesso accettabile.

I modelli in questione sono spesso di dimensioni relativamente piccole, e perciò possono essere eseguiti in locale. Anche se condividono la stessa architettura di base -- perché sono tutti transformer --, sono tipicamente addestrati per applicazioni specifiche.

Nel seguito mostriamo alcuni esempi, per ognuno dei quali usiamo un modello appropriato.

Per prima cosa importiamo i moduli Python che useremo e specifichiamo che, per semplicità, faremo eseguire il codice sempre in CPU anche nel caso una GPU sia disponibile.

In [1]:
from transformers import pipeline
from pprint import pprint

device = "cpu"

### 1. Sentiment analysis
https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment

In [3]:
model = 'nlptown/bert-base-multilingual-uncased-sentiment'
classifier = pipeline("sentiment-analysis", model=model, device=device)
res = classifier(
    "Ho trovato abbastanza interessante questa lezione sull'intelligenza artificiale",
    return_all_scores=True
)
pprint(res)

[[{'label': '1 star', 'score': 0.011834262870252132},
  {'label': '2 stars', 'score': 0.05826163291931152},
  {'label': '3 stars', 'score': 0.5082424879074097},
  {'label': '4 stars', 'score': 0.32383590936660767},
  {'label': '5 stars', 'score': 0.09782570600509644}]]


### 2. Zero-shot-classification
https://huggingface.co/Jiva/xlm-roberta-large-it-mnli (560M)

In [4]:
model = 'Jiva/xlm-roberta-large-it-mnli'
classifier = pipeline("zero-shot-classification", model=model, device=device)
res= classifier(
    "Ho trovato abbastanza interessante questa lezione su Alan Turing",
    candidate_labels=["economia", "geografia", "politica", "sport", "storia", "tecnologia"]
)
pprint(res)

{'labels': ['tecnologia',
            'storia',
            'geografia',
            'politica',
            'economia',
            'sport'],
 'scores': [0.40379834175109863,
            0.23238512873649597,
            0.10381027311086655,
            0.09817719459533691,
            0.08147850632667542,
            0.0803506076335907],
 'sequence': 'Ho trovato abbastanza interessante questa lezione su Alan Turing'}


### 3. Named Entity Recognition (NER)
https://huggingface.co/osiria/deberta-base-italian-uncased-ner

In [6]:
model = 'osiria/deberta-base-italian-uncased-ner'
ner = pipeline("ner", model=model, device=device, grouped_entities=True)
res = ner("Questa lezione è stata tenuta da Luca Mari per un corso master del Politecnico di Milano")
pprint(res)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'end': 42,
  'entity_group': 'PER',
  'score': np.float32(0.96193117),
  'start': 32,
  'word': 'Luca Mari'},
 {'end': 78,
  'entity_group': 'ORG',
  'score': np.float32(0.93260586),
  'start': 66,
  'word': 'Politecnico'},
 {'end': 88,
  'entity_group': 'LOC',
  'score': np.float32(0.91371405),
  'start': 81,
  'word': 'Milano'}]


### 4. Fill mask
https://huggingface.co/Musixmatch/umberto-wikipedia-uncased-v1

In [7]:
model = 'Musixmatch/umberto-wikipedia-uncased-v1'
unmasker = pipeline("fill-mask", model=model, device=device)
res = unmasker(
    "In questa lezione stiamo discutendo di <mask> artificiale.",
    top_k=3
)
pprint(res)

Some weights of the model checkpoint at Musixmatch/umberto-wikipedia-uncased-v1 were not used when initializing CamembertForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing CamembertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.7675500512123108,
  'sequence': 'n questa lezione stiamo discutendo di intelligenza artificiale.',
  'token': 12582,
  'token_str': 'intelligenza'},
 {'score': 0.043854448944330215,
  'sequence': 'n questa lezione stiamo discutendo di lingua artificiale.',
  'token': 1476,
  'token_str': 'lingua'},
 {'score': 0.010562906041741371,
  'sequence': 'n questa lezione stiamo discutendo di selezione artificiale.',
  'token': 7268,
  'token_str': 'selezione'}]


### 5. Question answering
https://huggingface.co/osiria/deberta-italian-question-answering (124M)

In [8]:
model = 'osiria/deberta-italian-question-answering'
question_answerer = pipeline("question-answering", model=model, device=device)
question_answerer(
    context="Stiamo tenendo questa lezione per un corso master del Politecnico di Milano",
    question="Per quale organizzazione si sta tenendo questa lezione?",
)

{'score': 0.9232677817344666,
 'start': 53,
 'end': 75,
 'answer': ' Politecnico di Milano'}

### 6. Text generation (completamento)
https://huggingface.co/microsoft/Phi-3.5-mini-instruct (3.82B)

In [9]:
model = "microsoft/Phi-3.5-mini-instruct"
generator = pipeline("text-generation", model=model, device=device)
res = generator(
    "In questa lezione sull'intelligenza artificiale, stiamo trattando di",
    max_length=50,
    truncation=True
)
pprint(res)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are not running the flash-attention implementation, expect numerical differences.


[{'generated_text': "In questa lezione sull'intelligenza artificiale, stiamo "
                    'trattando di un modello di apprendimento automatico '
                    'chiamato rete neurale. Una rete neurale è una struttura '
                    'computazionale ispir'}]


### 7. Text generation (Q&A)
https://huggingface.co/microsoft/Phi-3.5-mini-instruct (3.82B)

In [10]:
# https://huggingface.co/microsoft/Phi-3.5-mini-instruct (3.82B)
model = "microsoft/Phi-3.5-mini-instruct"
generator = pipeline("text-generation", model=model, device=device)
res = generator(
    [
        {
            "role": "system",
            "content": "Sei un esperto che risponde in italiano. e sempre in modo amichevole.",
        },
        {
            "role": "user",
            "content": """
                Fai una sintesi in due frasi del testo che segue?
                Il metodo scientifico ci ha insegnato a fare previsioni sulla base (1) di modelli dei fenomeni e (2) di dati: un modello ci dice che se il fenomeno è in una certa condizione, osserveremo certi effetti, e i dati ci dicono che siamo in una certa condizione; se tutto ciò è sufficientemente accurato, possiamo supporre che osserveremo proprio gli effetti predetti dal modello in base ai dati.
                Si tratta di una versione razionalizzata di ragionamenti che tutti noi, anche senza rendercene conto, attuiamo abitualmente: dall'idea, un modello appunto, che ci siamo fatti di come la nostra automobile si comporta quando freniamo e dai dati che otteniamo osservando intorno a noi mentre guidiamo, concludiamo se, quando e quanto intensamente dobbiamo schiacciare il freno.
                Ma in questo momento non abbiamo modelli affidabili di quello che sta succedendo a proposito dei sistemi cosiddetti di intelligenza artificiale generativa, e quindi questa strategia non è applicabile.
                Quando non abbiamo un modello a cui affidarci, a volte ricorriamo a una strategia di riserva: nel passato, in una certa condizione si sono prodotti certi effetti; se ora i dati ci dicono che siamo in una condizione simile, per analogia ipotizziamo che si produrranno effetti simili.
                Ragionando per analogia possiamo essere creativi, se troviamo nuove connessioni tra le entità a cui ci stiamo interessando, ma paradossalmente anche inerti di fronte ai cambiamenti, se ci ostiniamo a interpretare le novità alla luce di quello che già sappiamo.
                Ma anche questa strategia analogica non funziona oggi a proposito dei sistemi di intelligenza artificiale generativa: sia perché stiamo vivendo una condizione decisamente nuova, e quindi non sappiamo bene con cosa del passato confrontarla, sia perché le analogie sono valide quando i cambiamenti sono lineari, e invece i cambiamenti che stiamo vivendo in questi mesi sembrano proprio esponenziali.
                Insomma, se è spesso difficile fare previsioni affidabili senza modelli e dati affidabili, nella situazione in cui siamo potremmo ammettere onestamente che le nostre previsioni non hanno davvero nulla di affidabile.
            """
        }
    ],
    max_new_tokens=128
)
pprint(res)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[{'generated_text': [{'content': 'Sei un esperto che risponde in italiano. e '
                                 'sempre in modo amichevole.',
                      'role': 'system'},
                     {'content': '\n'
                                 '                Fai una sintesi in due frasi '
                                 'del testo che segue?\n'
                                 '                Il metodo scientifico ci ha '
                                 'insegnato a fare previsioni sulla base (1) '
                                 'di modelli dei fenomeni e (2) di dati: un '
                                 'modello ci dice che se il fenomeno è in una '
                                 'certa condizione, osserveremo certi effetti, '
                                 'e i dati ci dicono che siamo in una certa '
                                 'condizione; se tutto ciò è sufficientemente '
                                 'accurato, possiamo supporre che osserveremo '
    